In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
# Step 1: Define the dataset


movies1=pd.read_csv('tmdb_5000_movies.csv')
credits=pd.read_csv('tmdb_5000_credits.csv')

In [9]:
movies=movies1[['id','title','genres','overview','vote_average','tagline','keywords']]

In [10]:
import ast
def getgenre(obj):
    l=[]
    for i in ast.literal_eval( obj):
        l.append(i['name'])

    return l

In [11]:
movies['genres']=movies['genres'].apply(getgenre)
movies['keywords']=movies['keywords'].apply(getgenre)

C:\Users\JAHID\AppData\Local\Temp\ipykernel_3508\3838607355.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['genres']=movies['genres'].apply(getgenre)
C:\Users\JAHID\AppData\Local\Temp\ipykernel_3508\3838607355.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['keywords']=movies['keywords'].apply(getgenre)


In [12]:
credits["id"]=credits["movie_id"]
movies=movies.merge(credits,on='id')

In [13]:
def getgenreCast(obj):
    l=[]
    count=0
    for i in ast.literal_eval( obj):
        l.append(i['name'])
        count=count+1
        if(count>=10):
            break

    return " ".join(l)

In [14]:
movies['cast']=movies['cast'].apply(getgenreCast)
movies['cast'][0]

'Sam Worthington Zoe Saldana Sigourney Weaver Stephen Lang Michelle Rodriguez Giovanni Ribisi Joel David Moore CCH Pounder Wes Studi Laz Alonso'

In [15]:
def Director(obj):
    l=[]
    for i in ast.literal_eval( obj):
       if i['job']=='Director':
          l.append(i['name'])
          break

    return " ".join(l)


movies['crew']=movies['crew'].apply(Director)
movies['crew'][0]

'James Cameron'

In [16]:
movies.drop('title_y',  axis='columns', inplace=True)
movies.drop('movie_id',  axis='columns', inplace=True)
movies.rename(columns={"title_x":"title"},inplace=True)
movies['tagline']=movies['tagline'].fillna(value=' ')
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            4803 non-null   int64  
 1   title         4803 non-null   object 
 2   genres        4803 non-null   object 
 3   overview      4800 non-null   object 
 4   vote_average  4803 non-null   float64
 5   tagline       4803 non-null   object 
 6   keywords      4803 non-null   object 
 7   cast          4803 non-null   object 
 8   crew          4803 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 337.8+ KB


In [17]:
movies.head()

,id,title,genres,overview,vote_average,tagline,keywords,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","In the 22nd century, a paraplegic Marine is di...",7.2,Enter the World of Pandora.,"[culture clash, future, space war, space colon...",Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","Captain Barbossa, long believed to be dead, ha...",6.9,"At the end of the world, the adventure begins.","[ocean, drug abuse, exotic island, east india ...",Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski
2,206647,Spectre,"[Action, Adventure, Crime]",A cryptic message from Bond’s past sends him o...,6.3,A Plan No One Escapes,"[spy, based on novel, secret agent, sequel, mi...",Daniel Craig Christoph Waltz Léa Seydoux Ralph...,Sam Mendes
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]",Following the death of District Attorney Harve...,7.6,The Legend Ends,"[dc comics, crime fighter, terrorist, secret i...",Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan
4,49529,John Carter,"[Action, Adventure, Science Fiction]","John Carter is a war-weary, former military ca...",6.1,"Lost in our world, found in another.","[based on novel, mars, medallion, space travel...",Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton


In [18]:
movies['genres']=movies['genres'].apply(lambda x: " ".join(x))
movies['keywords']=movies['keywords'].apply(lambda x: " ".join(x))

In [19]:
movies.head()

,id,title,genres,overview,vote_average,tagline,keywords,cast,crew
0,19995,Avatar,Action Adventure Fantasy Science Fiction,"In the 22nd century, a paraplegic Marine is di...",7.2,Enter the World of Pandora.,culture clash future space war space colony so...,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron
1,285,Pirates of the Caribbean: At World's End,Adventure Fantasy Action,"Captain Barbossa, long believed to be dead, ha...",6.9,"At the end of the world, the adventure begins.",ocean drug abuse exotic island east india trad...,Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski
2,206647,Spectre,Action Adventure Crime,A cryptic message from Bond’s past sends him o...,6.3,A Plan No One Escapes,spy based on novel secret agent sequel mi6 bri...,Daniel Craig Christoph Waltz Léa Seydoux Ralph...,Sam Mendes
3,49026,The Dark Knight Rises,Action Crime Drama Thriller,Following the death of District Attorney Harve...,7.6,The Legend Ends,dc comics crime fighter terrorist secret ident...,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan
4,49529,John Carter,Action Adventure Science Fiction,"John Carter is a war-weary, former military ca...",6.1,"Lost in our world, found in another.",based on novel mars medallion space travel pri...,Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton


In [129]:
# Step 2: Combine relevant features into a single text field for each movie
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

def stemme(text):
    l=[]
    if type(text)==str:
        d=text.split()
        for i in d:
            l.append(ps.stem(i))
    else:
        l.append(text)

    if(len(l)>1):
        return " ".join(l)
    else:
        return text


movies['overview']=movies['overview'].apply(stemme)


In [130]:

def combine_features(movie):
    return f"{movie['title']} {movie['genres']} {movie['keywords']} {movie['cast']} {movie['crew']} {movie['tagline']} {movie['overview']}"
    # return f"{movie['title']}"
# movies_df = pd.DataFrame(movies)
# movies["combined_features"] = movies['title']
movies["combined_features"] =movies.apply(combine_features, axis=1)
# movies["combined_features"]=movies["combined_features"].apply(stemme)
movies["combined_features"]=movies["combined_features"].apply(lambda x:x.lower())
# movies['tags']=movies['tags'].astype(str)
# Step 3: Vectorize the combined features using TF-IDF
vectorizer = TfidfVectorizer(max_features=15000, stop_words="english")
tfidf_matrix = vectorizer.fit_transform( movies["combined_features"]).toarray()


In [113]:
stemme("Loving")

'love'

In [131]:
import pickle
pickle.dump(vectorizer,open('vectorizer.pkl','wb'))

pickle.dump(tfidf_matrix,open('tfidf_matrix.pkl','wb'))

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import pickle


In [ ]:

tfidf_matrix=pickle.load(open('tfidf_matrix.pkl','rb'))
vectorizer=pickle.load(open('vectorizer.pkl','rb'))
# vectorizer = TfidfVectorizer(max_features=5000, stop_words="english")
query="prison drama"
# query_vector = vectorizer.fit([query])
query_vector = vectorizer.transform([stemme(query).lower()])
list(enumerate(query_vector))
similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
# Get indices of top N similar movies
# similar_indices = similarities.argsort()[::-1][:5]
threshold = 0.3 # Define a similarity threshold
# sorted( similarities,reverse=True,key=lambda x:x[1])[0:6]
similar_indices = [i for i, similarity in enumerate(similarities) if similarity > threshold and movies.iloc[i].vote_average>5.3]
# similar_indices = similarities.argsort()[::-1]
    # Return the top N movies
for i in similar_indices:
    # if(similarities[i]>0):
    print(movies.iloc[i]["title"],movies.iloc[i].vote_average, similarities[i])

Escape Plan 6.7 0.31018678322708465
The Shawshank Redemption 8.5 0.46025943076354886
White Oleander 7.2 0.3280225508134906
Fortress 5.7 0.37668606335295246
American History X 8.2 0.3117606580530648
Standard Operating Procedure 6.6 0.30141321510116836
Mean Machine 6.0 0.35288414663936335
Prison 6.7 0.3588185160104939


In [133]:
stemme("Avatar")

'Avatar'

In [23]:

# Step 4: Define a function to search movies using a free-text query
def search_movies(query, top_n=3):
    # Vectorize the query
    query_vector = vectorizer.transform([query])
    # Calculate cosine similarity
    similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
    # Get indices of top N similar movies
    similar_indices = similarities.argsort()[::-1][:top_n]
    # Return the top N movies
    return [(movies.iloc[i]["title"], similarities[i]) for i in similar_indices]

# Step 5: Example usage
query = "Morgan Freeman prison drama"  # Free-text query
query = "A paraplegic Marine dispatched to the moon Pandora on a unique mission becomes torn between following his orders and protecting the world he feels is his home."  # Free-text query
query="Avater"
related_movies = search_movies(query.lower(), top_n=15)

# Output the results
print(f"Movies related to the query '{query}':")
for title, similarity in related_movies:
    print(f"{title} (Similarity: {similarity:.2f})")


    

NotFittedError: The TF-IDF vectorizer is not fitted